# Stabilized Collective Loan in Marlowe: Failure Due to Protocol Limits

This collective loan includes multiple investors and multiple borrowers, where the pool is supervised by an administrator and where a stabilization fund guarantees returns to investors, subject to a cap on losses.

Although this contract is simulatable without difficulty, ***this contract fails to execute*** on both the Preview testnet (due to its exceed the transaction-size limit) and the Pioneers testnet (due its execution memory and steps, even though this network has generously larger resources limits). (See Step 5 below for more details.) See [stabilized-collective-loan-success.ipynb](stabilized-collective-loan-success.ipynb) for an example of how including extra `Notify`s works around resource limits.

**Roles**
*   Role: Stabilizers
    *   Contributes funds to the stabilizer accounts of the contract.
    *   Stabilizer accounts reimburse Investors for shortfalls when the contract terminates.
        *   Reimbursement capped at the percentage stabilization of the loan pool.
    *   Stabilizers receive net excess loan proceeds when contract terminates.
*   Role: Investors
    *   Contribute liquidity used for lending.
    *   Guaranteed return of principal plus interest, but capped at percentage stabilization in case of severe net underperformance.
    *   May withdraw liquidity at any time.
*   Role: Administrator
    *   Approves loans.
    *   Decides when the contract terminates and settles to all parties.
*   Role: Borrowers
    *   Receives loans.
    *   Repays principal plus interest.

## Pool Operations

![Operations in the stabilized collective loan](stabilized-collective-loan-operations.png)

## Sequence Diagram

![Sequence of interactions in example](stabilized-collective-loan-interaction.svg)

## Preliminaries

### Versions

Make a note of the versions of the Marlowe repository and software.

In [1]:
export TREASURY=treasury
marlowe-cli --version

marlowe-cli 0.0.7.0 @ 25f6808de7aca1ab3d47db52fb4b66cae4228618


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

25f6808de7aca1ab3d47db52fb4b66cae4228618


### Select the Cardano network

Use the `preview` networks, which is in the Babbage era.

In [4]:
export CARDANO_TESTNET_MAGIC=1567
export CARDANO_NODE_SOCKET_PATH=/extra/iohk/networks/1567/node.socket
export CARDANO_ERA="--alonzo-era"

### Current Time

We'll start the contract at the current time.

In [5]:
NOW=$(($(date -u +%s) * 1000))
echo $NOW

1662329111000


Compute time constants, for convenience when setting time intervals for transactions.

In [6]:
MINUTE=$((60 * 1000))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

There is a minimum Ada requirement associated with a native token.

In [7]:
ADA=1000000
MIN_ADA=$((2 * ADA))
echo $MIN_ADA

2000000


### Contract Parameters

Set parameters for the contract. This contract scales to large numbers of lenders and borrowers, but that will involve many transactions and much time if run on-chain.

In [8]:
BASENAME=stabilized-collective-loan

DJED_POLICY=76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
DJED_NAME=DjedUSD

N_STABILIZERS=1
N_INVESTORS=2
N_BORROWERS=3

N_STEPS=20

MINIMUM_STABILIZATION=1000000
MAXIMUM_STABILIZATION=10000000

MINIMUM_INVESTMENT=500000
MAXIMUM_INVESTMENT=5000000

MINIMUM_LOAN=100000
MAXIMUM_LOAN=1000000

PERCENT_STABILIZATION=40
PERCENT_INTEREST=10

TIMEOUT_START=$((NOW + 1 * HOUR))
TIMEOUT_DELTA=$((1 * HOUR))
TIMEOUT_FINAL=$((NOW + 1000 * DAY))

ADMINISTRATOR_ROLE=A
STABILIZER_PREFIX=S
INVESTOR_PREFIX=I
BORROWER_PREFIX=B

### Helper functions for roles

In [9]:
function stabilizer() {
  echo "$STABILIZER_PREFIX$1"
}

In [10]:
function investor() {
  echo "$INVESTOR_PREFIX$1"
}

In [11]:
function borrower() {
  echo "$BORROWER_PREFIX$1"
}

### Helper functions for state

In [12]:
function show_state() {
  jq .tx.state $BASENAME-$1.marlowe | json2yaml
}

In [13]:
function show_choices() {
  jq .tx.state.choices $BASENAME-$1.marlowe | json2yaml
}

In [14]:
function show_nonchoices() {
  jq '.tx.state | {accounts : .accounts, boundValues : .boundValues}' $BASENAME-$1.marlowe | json2yaml
}

### Utility Functions

The following functions simplify UTxO management.

In [15]:
function find_role() {
  marlowe-cli "$CARDANO_ERA" util select --asset-only $ROLES_POLICY.$2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/'
}

In [16]:
function find_ada() {
  marlowe-cli "$CARDANO_ERA" util select --lovelace-only $((10 * ADA)) $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

In [17]:
function find_djed() {
  marlowe-cli "$CARDANO_ERA" util select --asset-only $DJED $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

### Setup the faucet.

In [18]:
FAUCET_SKEY_FILE=$TREASURY/payment.skey
FAUCET_ADDRESS=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDRESS"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Key Derivation

We'll derive all of the keys for stabilizers, investors, and borrowers from a single root private key.

In [19]:
ROOT_PRV=$TREASURY/william-shakespeare.root.prv

For the convenience of viewing transactions in Daedalus, we'll include the same stake credential in all of the addresses.

In [20]:
cardano-wallet key child 1852H/1815H/0H/2/0 < $ROOT_PRV \
| cardano-cli key convert-cardano-address-key --shelley-stake-key --signing-key-file /dev/stdin --out-file /dev/stdout \
| cardano-cli key verification-key --signing-key-file /dev/stdin --verification-key-file /dev/stdout \
| cardano-cli key non-extended-key --extended-verification-key-file /dev/stdin --verification-key-file $TREASURY/william-shakespeare.stake.vkey
cardano-cli stake-address build --testnet-magic $CARDANO_TESTNET_MAGIC --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey

stake_test1urplvp2a7dythh6yxutd0qlzzkncr3gy5ftvxj02d3etafqjugc5h


#### Key for the Administrator

Build the signing keys and payment addresses for the administrator from the `1852H/1815H/0H/0/` sequence of key derivations.

Use a pre-existing key for payment.

In [21]:
ADMINISTRATOR_SKEY=$TREASURY/christopher-marlowe.skey
ADMINISTRATOR_VKEY=$TREASURY/christopher-marlowe.vkey
ADMINISTRATOR_ADDR=$(cat $TREASURY/christopher-marlowe.testnet.address)
echo $ADMINISTRATOR_ADDR

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz


In [22]:
ADMINISTRATOR_SKEY[$j]=tmp/administrator.skey
ADMINISTRATOR_ADDR[$j]=$(
cardano-wallet key child 1852H/1815H/0H/0/0 < $ROOT_PRV \
| cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file $ADMINISTRATOR_SKEY
cardano-cli key verification-key --signing-key-file $ADMINISTRATOR_SKEY --verification-key-file /dev/stdout \
| cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
)
echo "Administrator 1852H/1815H/0H/0/0 = $ADMINISTRATOR_ADDR"

Administrator 1852H/1815H/0H/0/0 = addr_test1qp2l7afky3eqfkrht5f3qgy7x2yek5dejcnpnuqlwywz9twr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqp8jwfw


Send 100 Ada to the administrator.

In [23]:
marlowe-cli "$CARDANO_ERA" util faucet --lovelace "$((100 * ADA))"           \
                                       --faucet-address "$FAUCET_ADDRESS"    \
                                       --required-signer "$FAUCET_SKEY_FILE" \
                                       --out-file /dev/null                  \
                                       --submit 600                          \
                                       $ADMINISTRATOR_ADDR

TxId "94aafa594c2f6dc5124e7e754a25d08f33282e5f89a274644caa7d9af04f1506"


#### Keys for Stabilizers

Build the signing keys and payment addresses for the stabilizers from the `1852H/1815H/0H/0/` sequence of key derivations.

In [24]:
mkdir -p tmp
for j in `seq 1 $N_STABILIZERS`
do
  STABILIZER_SKEY[$j]=tmp/stabilizer-$j.skey
  STABILIZER_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$((1 + j)) < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${STABILIZER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${STABILIZER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
  )
  echo "Stabilizer $j = 1852H/1815H/0H/0/$((1 + j)) = ${STABILIZER_ADDR[$j]}"
done

Stabilizer 1 = 1852H/1815H/0H/0/2 = addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp


Send 25 ADA to each stabilizer.

In [25]:
marlowe-cli "$CARDANO_ERA" util faucet --lovelace "$((25 * ADA))"            \
                                       --faucet-address "$FAUCET_ADDRESS"    \
                                       --required-signer "$FAUCET_SKEY_FILE" \
                                       --out-file /dev/null                  \
                                       --submit 600                          \
                                       ${STABILIZER_ADDR[@]}

TxId "20a1716fcb85c55a085db88f5a23e3af6092aec0664f2fcf5c3047275067327f"


#### Keys for Investors

Build the signing keys and payment addresses for the investors from the `1852H/1815H/0H/0/` sequence of key derivations.

In [26]:
mkdir -p tmp
for j in `seq 1 $N_INVESTORS`
do
  INVESTOR_SKEY[$j]=tmp/investor-$j.skey
  INVESTOR_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$((1 + N_STABILIZERS + j)) < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${INVESTOR_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${INVESTOR_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
  )
  echo "Investor $j = 1852H/1815H/0H/0/$((1 + N_STABILIZERS + j)) = ${INVESTOR_ADDR[$j]}"
done

Investor 1 = 1852H/1815H/0H/0/3 = addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3
Investor 2 = 1852H/1815H/0H/0/4 = addr_test1qqjrjfslev50d5jn6qdeztxsqvuaaefq6e4hu55n59grjj7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqwhulyr


Send 25 ADA to each investor.

In [27]:
marlowe-cli "$CARDANO_ERA" util faucet --lovelace "$((25 * ADA))"            \
                                       --faucet-address "$FAUCET_ADDRESS"    \
                                       --required-signer "$FAUCET_SKEY_FILE" \
                                       --out-file /dev/null                  \
                                       --submit 600                          \
                                       ${INVESTOR_ADDR[@]}

TxId "98e1c65032305e535bd92205d97969f951e06613bd16b26291bd0c6d5dfa7879"


#### Keys for Borrowers

Build the signing keys and payment addresses for the borrowers from the `1852H/1815H/0H/0/` sequence of key derivations.

In [28]:
mkdir -p tmp
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_SKEY[$j]=tmp/borrower-$j.skey
  BORROWER_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$((1 + N_STABILIZERS + N_INVESTORS + j)) < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${BORROWER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${BORROWER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file $TREASURY/william-shakespeare.stake.vkey \
  )
  echo "Borrower $j = 1852H/1815H/0H/0/$((1 +N_STABILIZERS + N_INVESTORS + j)) = ${BORROWER_ADDR[$j]}"
done

Borrower 1 = 1852H/1815H/0H/0/5 = addr_test1qqxrzfv0v8l0lahyc5qsgpug66dafp8njl278g0pter7e3xr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqcjuuw0
Borrower 2 = 1852H/1815H/0H/0/6 = addr_test1qqqfzyuqkvrr7ajgk887sqac8lgzatc6el9uxke4x5g5dg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqkmrpwu
Borrower 3 = 1852H/1815H/0H/0/7 = addr_test1qp77epd4gy5hrt8pnh6h73se4durwrrkes3u0vj6tycwsq7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq7kyfpt


Send 25 ADA to each borrower.

In [29]:
marlowe-cli "$CARDANO_ERA" util faucet --lovelace "$((25 * ADA))"            \
                                       --faucet-address "$FAUCET_ADDRESS"    \
                                       --required-signer "$FAUCET_SKEY_FILE" \
                                       --out-file /dev/null                  \
                                       --submit 600                          \
                                       ${BORROWER_ADDR[@]}

TxId "8aa0e3ba72b15b5e52bf1f961e0f16a5334e969e21f5d6a09e52fa9a15acb8ad"


### Mint the Djed

The contract will use fake Djed for payments.

In [30]:
DJED_NAME=DjedUSD

In [31]:
TOTAL_DJED=$((
    N_STABILIZERS * MAXIMUM_STABILIZATION
  + N_INVESTORS   * MAXIMUM_INVESTMENT
  + N_BORROWERS   * MAXIMUM_LOAN
))
echo $TOTAL_DJED

23000000


In [32]:
DJED_POLICY=$(
marlowe-cli "$CARDANO_ERA" util mint --required-signer "$ADMINISTRATOR_SKEY" \
                                     --change-address "$ADMINISTRATOR_ADDR"  \
                                     --expires 1000000011                    \
                                     --count "$TOTAL_DJED"                   \
                                     --out-file /dev/null                    \
                                     --submit 600                            \
                                     "$DJED_NAME"                            \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                                         \
)
echo 'PolicyID "'$DJED_POLICY'"'

PolicyID "ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee"


In [33]:
DJED=$DJED_POLICY.$DJED_NAME
echo $DJED

ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.DjedUSD


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [34]:
echo $DJED_POLICY.$(echo -n $DJED_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344


### Distribute the Djed.

In [35]:
TXOUT=()
for j in `seq 1 $N_STABILIZERS`
do
  TXOUT+=("--tx-out" "${STABILIZER_ADDR[$j]}+$MIN_ADA+$MAXIMUM_STABILIZATION\ $DJED")
done
for j in `seq 1 $N_INVESTORS`
do
  TXOUT+=("--tx-out" "${INVESTOR_ADDR[$j]}+$MIN_ADA+$MAXIMUM_INVESTMENT\ $DJED")
done
for j in `seq 1 $N_BORROWERS`
do
  TXOUT+=("--tx-out" "${BORROWER_ADDR[$j]}+$MIN_ADA+$MAXIMUM_LOAN\ $DJED")
done

In [36]:
echo marlowe-cli "$CARDANO_ERA" transaction simple --tx-in "$(find_ada  $ADMINISTRATOR_ADDR)" \
                                                   --tx-in "$(find_djed $ADMINISTRATOR_ADDR)" \
                                                   ${TXOUT[@]}                                \
                                                   --change-address $ADMINISTRATOR_ADDR       \
                                                   --required-signer $ADMINISTRATOR_SKEY      \
                                                   --out-file /dev/null                       \
                                                   --submit 600                               \
| bash

TxId "f4778966d019f1290bf93fc4b34ec2d068e708680c5dc0f6e6252f89f03421c0"


### Mint the Role Tokens

In [37]:
for j in `seq 1 $N_STABILIZERS`
do
  STABILIZER_ROLE[$j]=$STABILIZER_PREFIX$j
done

In [38]:
for j in `seq 1 $N_INVESTORS`
do
  INVESTOR_ROLE[$j]=$INVESTOR_PREFIX$j
done

In [39]:
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_ROLE[$j]=$BORROWER_PREFIX$j
done

Mint the role tokens.

In [40]:
ROLES_POLICY=$(
marlowe-cli "$CARDANO_ERA" util mint --required-signer "$ADMINISTRATOR_SKEY" \
                                     --change-address "$ADMINISTRATOR_ADDR"  \
                                     --expires 1000000013                    \
                                     --out-file /dev/null                    \
                                     --submit 600                            \
                                     $ADMINISTRATOR_ROLE                     \
                                     ${STABILIZER_ROLE[@]}                   \
                                     ${INVESTOR_ROLE[@]}                     \
                                     ${BORROWER_ROLE[@]}                     \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                                       \
)
echo 'PolicyID "'$ROLES_POLICY'"'

PolicyID "2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f"


View the role tokens.

In [41]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMINISTRATOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c3eb80ff8048f3cd85adeaf6ed54e5ea5d3a31249fc20f4463040c386446ed74     0        17415145 lovelace + TxOutDatumNone
c3eb80ff8048f3cd85adeaf6ed54e5ea5d3a31249fc20f4463040c386446ed74     1        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.41 + TxOutDatumNone
c3eb80ff8048f3cd85adeaf6ed54e5ea5d3a31249fc20f4463040c386446ed74     2        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4231 + TxOutDatumNone
c3eb80ff8048f3cd85adeaf6ed54e5ea5d3a31249fc20f4463040c386446ed74     3        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4232 + TxOutDatumNone
c3eb80ff8048f3cd85adeaf6ed54e5ea5d3a31249fc20f4463040c386446ed74     4        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4233 + TxOutDatumNone
c

### Distribute the Role Tokens

In [42]:
TX_IN=($(find_ada  $ADMINISTRATOR_ADDR))
TXOUT=()
for j in `seq 1 $N_STABILIZERS`
do
  TXIN+=("--tx-in" "$(find_role $ADMINISTRATOR_ADDR ${STABILIZER_ROLE[$j]})")
  TXOUT+=("--tx-out" "${STABILIZER_ADDR[$j]}+$MIN_ADA+1\ $ROLES_POLICY.${STABILIZER_ROLE[$j]}")
done
for j in `seq 1 $N_INVESTORS`
do
  TXIN+=("--tx-in" "$(find_role $ADMINISTRATOR_ADDR ${INVESTOR_ROLE[$j]})")
  TXOUT+=("--tx-out" "${INVESTOR_ADDR[$j]}+$MIN_ADA+1\ $ROLES_POLICY.${INVESTOR_ROLE[$j]}")
done
for j in `seq 1 $N_BORROWERS`
do
  TXIN+=("--tx-in" "$(find_role $ADMINISTRATOR_ADDR ${BORROWER_ROLE[$j]})")
  TXOUT+=("--tx-out" "${BORROWER_ADDR[$j]}+$MIN_ADA+1\ $ROLES_POLICY.${BORROWER_ROLE[$j]}")
done

In [43]:
echo marlowe-cli "$CARDANO_ERA" transaction simple ${TXIN[@]}                              \
                                                   ${TXOUT[@]}                             \
                                                   --change-address "$ADMINISTRATOR_ADDR"  \
                                                   --required-signer "$ADMINISTRATOR_SKEY" \
                                                   --out-file /dev/null                    \
                                                   --submit 600                            \
| bash

TxId "ee45521486d46f3084ba41e157d73d18dc5cb95f9ecdafab22ca0f5d4b696fba"


View the role tokens.

In [44]:
echo "UTxOs at the Administor address:"
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMINISTRATOR_ADDR

UTxOs at the Administor address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c3eb80ff8048f3cd85adeaf6ed54e5ea5d3a31249fc20f4463040c386446ed74     0        17415145 lovelace + TxOutDatumNone
c3eb80ff8048f3cd85adeaf6ed54e5ea5d3a31249fc20f4463040c386446ed74     1        10000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.41 + TxOutDatumNone
ee45521486d46f3084ba41e157d73d18dc5cb95f9ecdafab22ca0f5d4b696fba     0        47774923 lovelace + TxOutDatumNone


In [45]:
for j in `seq 1 $N_STABILIZERS`
do
  echo "UTxOs at the Stabilizer $j address:"
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${STABILIZER_ADDR[$j]}
  echo
done

UTxOs at the Stabilizer 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
20a1716fcb85c55a085db88f5a23e3af6092aec0664f2fcf5c3047275067327f     2        25000000 lovelace + TxOutDatumNone
ee45521486d46f3084ba41e157d73d18dc5cb95f9ecdafab22ca0f5d4b696fba     1        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.5331 + TxOutDatumNone
f4778966d019f1290bf93fc4b34ec2d068e708680c5dc0f6e6252f89f03421c0     1        2000000 lovelace + 10000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone



In [46]:
for j in `seq 1 $N_INVESTORS`
do
  echo "UTxOs at the Investor $j address:"
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${INVESTOR_ADDR[$j]}
  echo
done

UTxOs at the Investor 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
98e1c65032305e535bd92205d97969f951e06613bd16b26291bd0c6d5dfa7879     2        25000000 lovelace + TxOutDatumNone
ee45521486d46f3084ba41e157d73d18dc5cb95f9ecdafab22ca0f5d4b696fba     2        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4931 + TxOutDatumNone
f4778966d019f1290bf93fc4b34ec2d068e708680c5dc0f6e6252f89f03421c0     2        2000000 lovelace + 5000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone

UTxOs at the Investor 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
98e1c65032305e535bd92205d97969f951e06613bd16b26291bd0c6d5dfa7879     3        25000000 lovelace + TxOutD

In [47]:
for j in `seq 1 $N_BORROWERS`
do
  echo "UTxOs at the Borrower $j address:"
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${BORROWER_ADDR[$j]}
  echo
done

UTxOs at the Borrower 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8aa0e3ba72b15b5e52bf1f961e0f16a5334e969e21f5d6a09e52fa9a15acb8ad     2        25000000 lovelace + TxOutDatumNone
ee45521486d46f3084ba41e157d73d18dc5cb95f9ecdafab22ca0f5d4b696fba     4        2000000 lovelace + 1 2cf679e1517d7171ba4a8a35d5ee9fb7f4d016249de49da74065a36f.4231 + TxOutDatumNone
f4778966d019f1290bf93fc4b34ec2d068e708680c5dc0f6e6252f89f03421c0     4        2000000 lovelace + 1000000 ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee.446a6564555344 + TxOutDatumNone

UTxOs at the Borrower 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8aa0e3ba72b15b5e52bf1f961e0f16a5334e969e21f5d6a09e52fa9a15acb8ad     3        25000000 lovelace + TxOutD

## Generate the Contract

Run a Haskell program that generates the contract.

In [48]:
runhaskell << EOI

{-# LANGUAGE NumericUnderscores #-}
{-# LANGUAGE OverloadedStrings  #-}
{-# LANGUAGE RecordWildCards    #-}


module Main (
  main
) where


import Control.Monad.Writer (Writer, runWriter, tell)
import Data.Aeson ((.=), encodeFile,object)
import Data.Default (Default(..))
import Data.Foldable (foldrM)
import Data.Maybe (fromMaybe)
import Data.String (fromString)
import Language.Marlowe.Core.V1.Semantics.Types
import Plutus.V1.Ledger.Api (BuiltinByteString, POSIXTime(..), TokenName)
import Language.Marlowe.CLI.Merkle (deepMerkleize)
import Language.Marlowe.CLI.Types (Continuations)


data Scenario =
  Scenario
  {
    nSteps               :: Int
  , timeoutStart         :: Integer
  , timeoutDelta         :: Integer
  , timeoutFinal         :: POSIXTime
  , nStabilizers         :: Int
  , nInvestors           :: Int
  , nBorrowers           :: Int
  , administratorName    :: String
  , stabilizerPrefix     :: String
  , investorPrefix       :: String
  , borrowerPrefix       :: String
  , stabilizeBound       :: [Bound]
  , investBound          :: [Bound]
  , loanBound            :: [Bound]
  , percentInterest      :: Integer
  , percentStabilization :: Integer
  , djed                 :: Token
  }

instance Default Scenario where
  def = 
    Scenario
    {
      nSteps               = $N_STEPS
    , timeoutStart         = $TIMEOUT_START
    , timeoutDelta         = $TIMEOUT_DELTA
    , timeoutFinal         = POSIXTime $TIMEOUT_FINAL
    , nStabilizers         = $N_STABILIZERS
    , nInvestors           = $N_INVESTORS
    , nBorrowers           = $N_BORROWERS
    , administratorName    = "$ADMINISTRATOR_ROLE"
    , stabilizerPrefix     = "$STABILIZER_PREFIX"
    , investorPrefix       = "$INVESTOR_PREFIX"
    , borrowerPrefix       = "$BORROWER_PREFIX"
    , stabilizeBound       = [Bound $MINIMUM_STABILIZATION $MAXIMUM_STABILIZATION]
    , investBound          = [Bound $MINIMUM_INVESTMENT    $MAXIMUM_INVESTMENT   ]
    , loanBound            = [Bound $MINIMUM_LOAN          $MAXIMUM_LOAN         ]
    , percentInterest      = $PERCENT_INTEREST
    , percentStabilization = $PERCENT_STABILIZATION
    , djed                 = Token "$DJED_POLICY" "$DJED_NAME"
    }


main :: IO ()
main =
  let
    (contract, continuations) = buildContract def
  in
    encodeFile "$BASENAME.merkleization"
      $ object
        [
          "contract"      .= contract
        , "continuations" .= continuations
        ]


buildContract :: Scenario -> (Contract, Continuations)
buildContract scenario@Scenario{..} =
  let
    timeouts = [POSIXTime $ timeoutStart + toInteger i * timeoutDelta | i <- [1..nSteps]]
  in
    runWriter $ foldrM (makeStep scenario) Close timeouts


defaultBound :: [Bound]
defaultBound = [Bound 1 1_000_000_000_000]


makeParty :: String -> Int -> Party
makeParty name i = Role . fromString $ name <> show i


deposit :: Scenario
        -> String
        -> String
        -> Int
        -> Maybe Party
        -> [Bound]
        -> (Value Observation -> Contract -> Contract)
        -> Contract
        -> [Case Contract]
deposit Scenario{..} name prefix i recipient bound f continuation =
  let
    party = makeParty prefix i
    party' = fromMaybe party recipient
    choice = ChoiceId (fromString name) party
    value = ChoiceValue choice
  in
    [
      Case (Choice choice bound)
        continuation
    , Case (Deposit party' party djed value)
        $ f value continuation
    ]


stabilization :: ValueId
stabilization = ValueId "s"


stabilize :: Scenario -> Int -> Contract -> [Case Contract]
stabilize scenario@Scenario{..} i =
  deposit scenario "S" stabilizerPrefix i Nothing stabilizeBound
    (stabilization =+)
 

assets :: ValueId
assets = ValueId "a"


equity :: Int -> ValueId
equity = ValueId . ("e" <>) . fromString . show


invest :: Scenario -> Int -> Contract -> [Case Contract]
invest scenario@Scenario{..} i =
  deposit scenario "I" investorPrefix i Nothing investBound
    $ \value -> (assets   =+ value)
              . (equity i =+ value)


withdraw :: Scenario
         -> Int
         -> Contract
         -> [Case Contract]
withdraw Scenario{..} i continuation =
  let
    party = makeParty investorPrefix i
    choice = ChoiceId "W" party
    value = ChoiceValue choice
    funds = AvailableMoney party djed
  in
    [
      Case (Choice choice defaultBound)
        $ If (value @<= funds)
            (
              Pay party (Party party) djed value
                $ assets   =- value
                $ equity i =- value
                $ continuation
            )
            (
              Pay party (Party party) djed funds
                $ assets   =- funds
                $ equity i =- funds
                $ continuation
            )
    ]


lend :: Scenario
     -> Party
     -> Value Observation
     -> Value Observation
     -> Int
     -> Contract
     -> Contract
lend Scenario{..} borrower principal interest i =
  let
    party = makeParty investorPrefix i
    funds = AvailableMoney party djed
    assets' = UseValue assets
    principal' = (principal @* funds) @/ assets'
    interest'  = (interest  @* funds) @/ assets'
  in
    Pay party (Party borrower) djed principal'
      . (equity i =+ interest')


liabilities :: ValueId
liabilities = ValueId "l"

loan :: Scenario
     -> Int
     -> Contract
     -> [Case Contract]
loan scenario@Scenario{..} i continuation =
  let
    party = Role $ fromString administratorName
    borrower = makeParty borrowerPrefix i
    choice = ChoiceId (fromString $ "L" <> show i) party
    principal = ChoiceValue choice
    assets' = UseValue assets
    stabilization' = UseValue stabilization
  in
    [
      Case (Choice choice loanBound)
        $ let
            interest = principal @% percentInterest
            debt = principal @+ interest
          in
            If ((principal @<= assets') @&& (stabilization' @>= (debt @% percentStabilization)))
              (
                flip (foldr $ lend scenario borrower principal interest) [1..nInvestors]
                  $ liabilities =+ debt
                  $ assets      =- principal
                  $ continuation
              )
              continuation
    ]


restore :: Scenario
        -> Value Observation
        -> Int
        -> Contract
        -> Contract
restore Scenario{..} repayment i =
  let
    administrator = Role $ fromString administratorName
    party = makeParty investorPrefix i
    equity' = UseValue $ equity i
    assets' = AvailableMoney party djed
    liabilities' = UseValue liabilities
    share = (repayment @* (equity' @- assets')) @/ liabilities'
  in
    Pay administrator (Account party) djed share


repay :: Scenario
      -> Int
      -> Contract
      -> [Case Contract]
repay scenario@Scenario{..} i =
  deposit scenario "R" borrowerPrefix i (Just . Role $ fromString administratorName) defaultBound
    $ \repayment -> flip (foldr $ restore scenario repayment) [1..nInvestors]
                      . (liabilities =- repayment)
                      . (assets      =+ repayment)


gather :: Scenario
       -> Value Observation
       -> ValueId
       -> String
       -> Int
       -> Contract
       -> Contract
gather Scenario{..} amount basis prefix i = 
  let
    administrator = Role $ fromString administratorName
    party = makeParty prefix i
    funds = AvailableMoney party djed
    basis' = UseValue basis
    share = (amount @* funds) @/ basis'
  in
    Pay party (Account administrator) djed share


scatter :: Scenario
        -> ValueId
        -> String
        -> Int
        -> Contract
        -> Contract
scatter Scenario{..} basis prefix i =
  let
    administrator = Role $ fromString administratorName
    party = makeParty prefix i
    funds = AvailableMoney party djed
    basis' = UseValue basis
    amount = AvailableMoney administrator djed
    share = (amount @* funds) @/ basis'
  in
    Pay administrator (Account party) djed share


terminate :: Scenario
          -> [Case Contract]
terminate scenario@Scenario{..} =
  let
    party = Role $ fromString administratorName
    choice = ChoiceId "T" party
    liabilities' = UseValue liabilities
  in
    [
      Case (Choice choice [Bound 1 1])
        $ If (liabilities' @>= Constant 0)
          (
              flip (foldr $ gather  scenario liabilities' stabilization stabilizerPrefix) [1..nStabilizers]
            $ flip (foldr $ scatter scenario              assets        investorPrefix  ) [1..nInvestors  ]
              Close
          )
          (
              flip (foldr $ gather  scenario (NegValue liabilities') assets        investorPrefix  ) [1..nInvestors  ]
            $ flip (foldr $ scatter scenario                         stabilization stabilizerPrefix) [1..nStabilizers]
              Close
          )
    ]


makeStep :: Scenario
         -> Timeout
         -> Contract
         -> Writer Continuations Contract
makeStep scenario@Scenario{..} timeout continuation =
  let
  in
    deepMerkleize
      . When
        (
          concat
            $  [stabilize scenario i continuation | i <- [1..nStabilizers]]
            <> [invest    scenario i continuation | i <- [1..nInvestors]  ]
            <> [withdraw  scenario i continuation | i <- [1..nInvestors]  ]
            <> [loan      scenario i continuation | i <- [1..nBorrowers]  ]
            <> [repay     scenario i continuation | i <- [1..nBorrowers]  ]
            <> [terminate scenario                                        ]
        )
      timeout
    =<< merkleizeTimeout timeoutFinal continuation


merkleizeTimeout :: Timeout
                 -> Contract
                 -> Writer Continuations Contract
merkleizeTimeout timeout continuation =
  deepMerkleize
    $ When
      [
        Case (Notify TrueObs )
        continuation
      ]
      timeout
      Close


(=+) :: ValueId -> Value Observation -> (Contract -> Contract)
variable =+ value = Let variable $ UseValue variable @+ value

(=-) :: ValueId -> Value Observation -> (Contract -> Contract)
variable =- value = Let variable $ UseValue variable @- value

(=:) :: ValueId -> Integer -> (Contract -> Contract)
variable =: value = Let variable $ Constant value

(@%) :: Value Observation -> Integer -> Value Observation
value @% rate = DivValue (MulValue value $ Constant rate) (Constant 100)

(@+%) :: Value Observation -> Integer -> Value Observation
value @+% rate = DivValue (MulValue value (AddValue (Constant rate) $ Constant 0)) (Constant 100)

(@+) :: Value Observation -> Value Observation -> Value Observation
x @+ y =AddValue x y 

(@-) :: Value Observation -> Value Observation -> Value Observation
x @- y = SubValue x y

(@*) :: Value Observation -> Value Observation -> Value Observation
x @* y = MulValue x y

(@/) :: Value Observation -> Value Observation -> Value Observation
x @/ y = DivValue x y

(@<) :: Value Observation -> Value Observation -> Observation
x @< y = ValueLT x y

(@<=) :: Value Observation -> Value Observation -> Observation
x @<= y = ValueLE x y

(@>) :: Value Observation -> Value Observation -> Observation
x @> y = ValueGT x y

(@>=) :: Value Observation -> Value Observation -> Observation
x @>= y = ValueGE x y

(@==) :: Value Observation -> Value Observation -> Observation
x @== y = ValueEQ x y

(@&&) :: Observation -> Observation -> Observation
x @&& y = AndObs x y

(@||) :: Observation -> Observation -> Observation
x @|| y = OrObs x y

EOI

How large is the contract?

In [49]:
ls -lh $BASENAME.merkleization

-rw-rw-r-- 1 bbush bbush-upg 1.9M Sep  4 16:11 stabilized-collective-loan.merkleization


How many `Case` statements does it contain?

In [50]:
sed -e 's/case/&\n/g' $BASENAME.merkleization | grep case | wc -l

6156


How many merkleizations does it contain?

In [51]:
jq '.continuations | length' $BASENAME.merkleization

240


Separate the contract from the merkleizations.

In [52]:
jq .contract $BASENAME.merkleization > $BASENAME-1.contract

## Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [53]:
yaml2json << EOI > $BASENAME-1.state
accounts:
- - - role_token: $ADMINISTRATOR_ROLE
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [54]:
cat $BASENAME-1.state

{"accounts":[[[{"role_token":"A"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1662329111000}


## Simulate and Execute the Contract

### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract. Note that we merkleization the contract because of its large size.

In [55]:
n=1

In [56]:
marlowe-cli "$CARDANO_ERA" run initialize --roles-currency "$ROLES_POLICY"        \
                                          --contract-file "$BASENAME-$n.contract" \
                                          --state-file    "$BASENAME-$n.state"    \
                                          --out-file      "$BASENAME-$n.partial"  \
                                          --print-stats


Validator size: 12412
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18768100, exBudgetMemory = ExMemory 81700}


Manually merge the continuations into the `.marlowe` file.

In [57]:
jq -s '.[0] * {tx : .[1]}' $BASENAME-1.partial $BASENAME.merkleization > $BASENAME-1.marlowe

The minimum Ada deposit has been recorded.

In [58]:
show_state $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1662329111000


In [59]:
DEADLINE=$((NOW + 59 * MINUTE))

Submit the transaction.

In [60]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-out-file "$BASENAME-$n.marlowe" \
                                            --required-signer "$ADMINISTRATOR_SKEY"   \
                                            --change-address "$ADMINISTRATOR_ADDR"    \
                                            --out-file "$BASENAME-$n.raw"             \
                                            --print-stats                             \
                                            --submit=600                              \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                      \
)


Fee: Lovelace 257593
Size: 1951 / 32768 = 5%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%


In [61]:
echo "Transaction $n hash: ${TX_ID[$n]}"

Transaction 1 hash: 9fa6d721091f1e76028b52754cda002f55e3e5283674a819937584a7e6931715


### 2. Stabilzer 1 deposits 5M Djed

#### Choose the deposit

In [62]:
STABILIZER_DEPOSIT=5000000

In [63]:
n=$((n+1))

In [64]:
marlowe-cli run prepare --choice-party "Role=$(stabilizer 1)"            \
                        --choice-name S                                  \
                        --choice-number "$STABILIZER_DEPOSIT"            \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Datum size: 1676


In [65]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000


Submit the transaction.

In [66]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                                            --tx-in-marlowe "${TX_ID[$((n-1))]}#1"         \
                                            --marlowe-out-file "$BASENAME-$n.marlowe"      \
                                            --required-signer "${STABILIZER_SKEY[1]}"      \
                                            --change-address "${STABILIZER_ADDR[1]}"       \
                                            --out-file "$BASENAME-$n.raw"                  \
                                            --print-stats                                  \
                                            --submit=600                                   \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                           \
)


Fee: Lovelace 1888587
Size: 18018 / 32768 = 54%
Execution units:
  Memory: 10592836 / 30000000 = 35%
  Steps: 4276104505 / 10000000000 = 42%


In [67]:
echo "Transaction $n hash: ${TX_ID[$n]}"

Transaction 2 hash: e7497c21e133c7c259836e835c60897e9e5c5616724f3a0469ace436ffb5990d


#### Make the deposit

In [68]:
n=$((n+1))

In [69]:
marlowe-cli run prepare --deposit-account Role=$(stabilizer 1)         \
                        --deposit-party Role=$(stabilizer 1)           \
                        --deposit-token $DJED                          \
                        --deposit-amount $STABILIZER_DEPOSIT           \
                        --invalid-before $NOW                          \
                        --invalid-hereafter $((DEADLINE - 1 * MINUTE)) \
                        --marlowe-file $BASENAME-$((n-1)).marlowe      \
                        --out-file     $BASENAME-$n.marlowe            \
                        --print-stats


Datum size: 1745


In [70]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
boundValues:
- - s
  - 5000000


Submit the transaction.

In [71]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                                            --tx-in-marlowe "${TX_ID[$((n-1))]}#1"         \
                                            --marlowe-out-file "$BASENAME-$n.marlowe"      \
                                            --required-signer "${STABILIZER_SKEY[1]}"      \
                                            --change-address "${STABILIZER_ADDR[1]}"       \
                                            --out-file "$BASENAME-$n.raw"                  \
                                            --print-stats                                  \
                                            --submit=600                                   \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                           \
)


Fee: Lovelace 2122253
Size: 18388 / 32768 = 56%
Execution units:
  Memory: 13185078 / 30000000 = 43%
  Steps: 5155019385 / 10000000000 = 51%


In [72]:
echo "Transaction $n hash: ${TX_ID[$n]}"

Transaction 3 hash: 4e11c9ed175e34695eedec4e0a2afb9ba869c0e8f1dcbee91ccbd294f0f060c5


### 3. Investor 1 deposits 1M Djed

#### Choose the deposit

In [73]:
INVESTOR_DEPOSIT=1000000

In [74]:
n=$((n+1))

In [75]:
marlowe-cli run prepare --choice-party "Role=$(investor 1)"              \
                        --choice-name I                                  \
                        --choice-number "$INVESTOR_DEPOSIT"              \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Datum size: 1763


In [76]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000


Submit the transaction.

In [77]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                                            --tx-in-marlowe "${TX_ID[$((n-1))]}#1"         \
                                            --marlowe-out-file "$BASENAME-$n.marlowe"      \
                                            --required-signer "${INVESTOR_SKEY[1]}"        \
                                            --change-address "${INVESTOR_ADDR[1]}"         \
                                            --out-file "$BASENAME-$n.raw"                  \
                                            --print-stats                                  \
                                            --submit=600                                   \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                           \
)


Fee: Lovelace 1996160
Size: 18242 / 32768 = 55%
Execution units:
  Memory: 11802372 / 30000000 = 39%
  Steps: 4663433556 / 10000000000 = 46%


In [78]:
echo "Transaction $n hash: ${TX_ID[$n]}"

Transaction 4 hash: b18095d55872b0522c164e3129f2298ddb438cfcc0b723d5fbf37e2e8e4b273c


#### Make the deposit

In [79]:
n=$((n+1))

In [80]:
marlowe-cli run prepare --deposit-account "Role=$(investor 1)"           \
                        --deposit-party "Role=$(investor 1)"             \
                        --deposit-token "$DJED"                          \
                        --deposit-amount "$INVESTOR_DEPOSIT"             \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Datum size: 1844


In [81]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 1000000
boundValues:
- - s
  - 5000000
- - a
  - 1000000
- - e1
  - 1000000


Submit the transaction.

In [82]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                                            --tx-in-marlowe "${TX_ID[$((n-1))]}#1"         \
                                            --marlowe-out-file "$BASENAME-$n.marlowe"      \
                                            --required-signer "${INVESTOR_SKEY[1]}"        \
                                            --change-address "${INVESTOR_ADDR[1]}"         \
                                            --out-file "$BASENAME-$n.raw"                  \
                                            --print-stats                                  \
                                            --submit=600                                   \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                           \
)


Fee: Lovelace 2260830
Size: 18619 / 32768 = 56%
Execution units:
  Memory: 14739498 / 30000000 = 49%
  Steps: 5692084207 / 10000000000 = 56%


In [83]:
echo "Transaction $n hash: ${TX_ID[$n]}"

Transaction 5 hash: 2051ffd55ff03ea057d68d09b625c3742fbbb75ae3d20d2c6286861a2eeaf343


### 4. Investor 2 deposits 500k Djed

#### Choose the deposit

In [84]:
INVESTOR_DEPOSIT=500000

In [85]:
n=$((n+1))

In [86]:
marlowe-cli run prepare --choice-party "Role=$(investor 2)"              \
                        --choice-name I                                  \
                        --choice-number "$INVESTOR_DEPOSIT"              \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Datum size: 1862


In [87]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: I
    choice_owner:
      role_token: I2
  - 500000


Submit the transaction.

In [88]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                                            --tx-in-marlowe "${TX_ID[$((n-1))]}#1"         \
                                            --marlowe-out-file "$BASENAME-$n.marlowe"      \
                                            --required-signer "${INVESTOR_SKEY[2]}"        \
                                            --change-address "${INVESTOR_ADDR[2]}"         \
                                            --out-file "$BASENAME-$n.raw"                  \
                                            --print-stats                                  \
                                            --submit=600                                   \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                           \
)


Fee: Lovelace 2050836
Size: 18440 / 32768 = 56%
Execution units:
  Memory: 12362620 / 30000000 = 41%
  Steps: 4852589066 / 10000000000 = 48%


In [89]:
echo "Transaction $n hash: ${TX_ID[$n]}"

Transaction 6 hash: 6ef31eeacc8e7bf42b5e50328c61731a51d7895cce794387abf021c80740ee28


#### Make the deposit

In [90]:
n=$((n+1))

In [91]:
marlowe-cli run prepare --deposit-account "Role=$(investor 2)"           \
                        --deposit-party "Role=$(investor 2)"             \
                        --deposit-token "$DJED"                          \
                        --deposit-amount "$INVESTOR_DEPOSIT"             \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Warnings:
  TransactionShadowing "a" 1000000 1500000
Datum size: 1932


In [92]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 1000000
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 500000
boundValues:
- - s
  - 5000000
- - a
  - 1500000
- - e1
  - 1000000
- - e2
  - 500000


Submit the transaction.

In [93]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                                            --tx-in-marlowe "${TX_ID[$((n-1))]}#1"         \
                                            --marlowe-out-file "$BASENAME-$n.marlowe"      \
                                            --required-signer "${INVESTOR_SKEY[2]}"        \
                                            --change-address "${INVESTOR_ADDR[2]}"         \
                                            --out-file "$BASENAME-$n.raw"                  \
                                            --print-stats                                  \
                                            --submit=600                                   \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                           \
)


Fee: Lovelace 2336415
Size: 18806 / 32768 = 57%
Execution units:
  Memory: 15568666 / 30000000 = 51%
  Steps: 5962734533 / 10000000000 = 59%


In [94]:
echo "Transaction $n hash: ${TX_ID[$n]}"

Transaction 7 hash: 9bf0d07e5cc2fbf4de3e330811b2d349c982db8fdef1877f1884271edcf35dfe


### 5. Administrator authorizes loan of 900k Djed to Borrower 1

In [95]:
LOAN_AMOUNT=900000

In [96]:
n=$((n+1))

In [97]:
marlowe-cli run prepare --choice-party "Role=$ADMINISTRATOR_ROLE"        \
                        --choice-name L1                                 \
                        --choice-number "$LOAN_AMOUNT"                   \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Warnings:
  TransactionShadowing "e1" 1000000 1024000
  TransactionShadowing "e2" 500000 512000
  TransactionShadowing "a" 1500000 600000
Datum size: 1961
Payment 1
  Acccount: "I1"
  Payee: Party "B1"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 600000
Payment 2
  Acccount: "I2"
  Payee: Party "B1"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 300000


In [98]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 400000
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 200000
boundValues:
- - s
  - 5000000
- - a
  - 600000
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 990000


Submit the transaction.

In [99]:
TX_ID[$n]=$(
marlowe-cli "$CARDANO_ERA" run auto-execute --marlowe-in-file "$BASENAME-$((n-1)).marlowe" \
                                            --tx-in-marlowe "${TX_ID[$((n-1))]}#1"         \
                                            --marlowe-out-file "$BASENAME-$n.marlowe"      \
                                            --required-signer "$ADMINISTRATOR_SKEY"        \
                                            --change-address "$ADMINISTRATOR_ADDR"         \
                                            --out-file "$BASENAME-$n.raw"                  \
                                            --print-stats                                  \
                                            --submit=600                                   \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                                           \
)


Fee: Lovelace 3973550
Size: 21204 / 32768 = 64%
Execution units:
  Memory: 33595585 / 30000000 = 111%
  Steps: 12840883044 / 10000000000 = 128%
TxValidationErrorInMode (ShelleyTxValidationError ShelleyBasedEraAlonzo (ApplyTxError [UtxowFailure (WrappedShelleyEraFailure (UtxoFailure (ExUnitsTooBigUTxO (WrapExUnits {unWrapExUnits = ExUnits' {exUnitsMem' = 30000000, exUnitsSteps' = 10000000000}}) (WrapExUnits {unWrapExUnits = ExUnits' {exUnitsMem' = 33595585, exUnitsSteps' = 12840883044}}))))])) AlonzoEraInCardanoMode


***This transaction fails because it exceeds the protocol limits both for execution memory and for execution steps.***

The complexity of the following `Action` is the main reason that this step exceeds memory and step limits. A minor contributor is the numerous `Actions` in the `When` term.

![Loan authorization exceeds protocol parameters.](stabilized-collective-loan-withdraw.png)

*Let continue simulating the contract, even though we cannot execute it on the blockchain.*

### 6. Administrator authorizes loan of 600k Djed to Borrower 2

In [100]:
LOAN_AMOUNT=600000

In [101]:
n=$((n+1))

In [102]:
marlowe-cli run prepare --choice-party Role="$ADMINISTRATOR_ROLE"        \
                        --choice-name L2                                 \
                        --choice-number "$LOAN_AMOUNT"                   \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Warnings:
  TransactionShadowing "e1" 1024000 1024000
  TransactionShadowing "e2" 512000 512000
  TransactionShadowing "l" 990000 1650000
  TransactionShadowing "a" 600000 0
Datum size: 1859
Payment 1
  Acccount: "I1"
  Payee: Party "B2"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 400000
Payment 2
  Acccount: "I2"
  Payee: Party "B2"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 200000


In [103]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
boundValues:
- - s
  - 5000000
- - a
  - 0
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 1650000


### 7. Borrower 2 pays back 720k Djed owed

#### Choose the deposit

In [104]:
BORROWER_REPAYMENT=720000

In [105]:
n=$((n+1))

In [106]:
marlowe-cli run prepare --choice-party "Role=$(borrower 2)"              \
                        --choice-name R                                  \
                        --choice-number "$BORROWER_REPAYMENT"            \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Datum size: 1877


In [107]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: I
    choice_owner:
      role_token: I2
  - 500000
- - choice_name: L1
    choice_owner:
      role_token: A
  - 900000
- - choice_name: L2
    choice_owner:
      role_token: A
  - 600000
- - choice_name: R
    choice_owner:
      role_token: B2
  - 720000


#### Make the deposit

In [108]:
n=$((n+1))

In [109]:
marlowe-cli run prepare --deposit-account "Role=$ADMINISTRATOR_ROLE"     \
                        --deposit-party "Role=$(borrower 2)"             \
                        --deposit-token "$DJED"                          \
                        --deposit-amount "$BORROWER_REPAYMENT"           \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Warnings:
  TransactionShadowing "l" 1650000 930000
  TransactionShadowing "a" 0 720000
Datum size: 2052
Payment 1
  Acccount: "A"
  Payee: Account "I1"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 446836
Payment 2
  Acccount: "A"
  Payee: Account "I2"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 223418


In [110]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: A
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 49746
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 446836
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 223418
boundValues:
- - s
  - 5000000
- - a
  - 720000
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 930000


### 8. Borrower 1 only pays back 840k of the 990k Djed owed

#### Choose the deposit

In [111]:
BORROWER_REPAYMENT=840000

In [112]:
n=$((n+1))

In [113]:
marlowe-cli run prepare --choice-party "Role=$(borrower 1)"              \
                        --choice-name R                                  \
                        --choice-number "$BORROWER_REPAYMENT"            \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Datum size: 2070


In [114]:
show_choices $n

- - choice_name: S
    choice_owner:
      role_token: S1
  - 5000000
- - choice_name: I
    choice_owner:
      role_token: I1
  - 1000000
- - choice_name: I
    choice_owner:
      role_token: I2
  - 500000
- - choice_name: L1
    choice_owner:
      role_token: A
  - 900000
- - choice_name: L2
    choice_owner:
      role_token: A
  - 600000
- - choice_name: R
    choice_owner:
      role_token: B2
  - 720000
- - choice_name: R
    choice_owner:
      role_token: B1
  - 840000


### Make the deposit

In [115]:
n=$((n+1))

In [116]:
marlowe-cli run prepare --deposit-account "Role=$ADMINISTRATOR_ROLE"     \
                        --deposit-party "Role=$(borrower 1)"             \
                        --deposit-token "$DJED"                          \
                        --deposit-amount "$BORROWER_REPAYMENT"           \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Warnings:
  TransactionShadowing "l" 930000 90000
  TransactionShadowing "a" 720000 1560000
Datum size: 2072
Payment 1
  Acccount: "A"
  Payee: Account "I1"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 521309
Payment 2
  Acccount: "A"
  Payee: Account "I2"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 260654


In [117]:
show_nonchoices $n

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: S1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 5000000
- - - role_token: A
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 107783
- - - role_token: I1
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 968145
- - - role_token: I2
    - currency_symbol: ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee
      token_name: DjedUSD
  - 484072
boundValues:
- - s
  - 5000000
- - a
  - 1560000
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 90000


### 7. Administrator closes the pool

In [118]:
n=$((n+1))

In [119]:
marlowe-cli run prepare --choice-party "Role=$ADMINISTRATOR_ROLE"        \
                        --choice-name T                                  \
                        --choice-number 1                                \
                        --invalid-before "$NOW"                          \
                        --invalid-hereafter "$((DEADLINE - 1 * MINUTE))" \
                        --marlowe-file "$BASENAME-$((n-1)).marlowe"      \
                        --out-file     "$BASENAME-$n.marlowe"            \
                        --print-stats


Datum size: 221
Payment 1
  Acccount: "S1"
  Payee: Account "A"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 90000
Payment 2
  Acccount: "A"
  Payee: Account "I1"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 122745
Payment 3
  Acccount: "A"
  Payee: Account "I2"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 23284
Payment 4
  Acccount: "A"
  Payee: Party "A"
  Ada: 2.000000
Payment 5
  Acccount: "S1"
  Payee: Party "S1"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 4910000
Payment 6
  Acccount: "A"
  Payee: Party "A"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 51754
Payment 7
  Acccount: "I1"
  Payee: Party "I1"
  Ada: 0.000000
  ea94ed18f7a5bf2199f638360443a31ab3ec639108dde4c9e3e9bfee."DjedUSD": 1090890
Payment 8
  Acccount: "I2"
  Payee: Party "I2"
  Ada: 0.000000
  ea94ed18f7a5bf219

In [120]:
show_nonchoices $n

accounts: []
boundValues:
- - s
  - 5000000
- - a
  - 1560000
- - e1
  - 1024000
- - e2
  - 512000
- - l
  - 90000
